In [1]:
import pickle  # pickle 모듈을 import하여 바이너리 형식으로 저장된 데이터를 읽고 쓸 수 있도록 함

with open('./res/rag-custom.pkl', 'rb') as f:  # 바이너리 읽기 모드로 'rag_data.pkl' 파일을 엶
    rag_data = pickle.load(f)  # 파일에서 데이터를 읽어와 'rag_data' 변수에 저장

In [2]:
questions = rag_data['questions'][:10]  # 상위 10개의 질문을 'questions'에 저장
contexts = rag_data['contexts'][:10]  # 상위 10개의 문맥을 'contexts'에 저장
answers = rag_data['answers'][:10]  # 상위 10개의 정답을 'answers'에 저장
len(questions)  # 'questions' 리스트의 길이(질문 수)를 반환

10

In [3]:
from ragas.llms import LangchainLLMWrapper  # Langchain LLM을 래핑하는 클래스 import
from langchain_openai import ChatOpenAI  # OpenAI와의 연동을 위한 Langchain 기반 클래스 import
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))  # GPT-4o 모델을 사용해 평가 LLM 인스턴스 생성

In [4]:
from tqdm import tqdm  # 진행 상태 표시를 위한 tqdm 모듈 import

from utils import call_openai, get_embeddings, cosine_similarity  # 사용자 정의 모듈에서 필요한 함수 import

predictions = []  # 예측 결과를 저장할 리스트 초기화
for i in tqdm(range(len(questions))):  # 질문 수 만큼 반복하며 진행 상태 표시
    prompt = f"""You are an expert in Finance. Please answer to the question given below.
    
Question:
{questions[i]}
"""  # 금융 분야 전문가로 가정하여 질문에 답변하도록 하는 프롬프트 생성

    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')  # OpenAI API 호출을 통해 예측 생성
    predictions.append(prediction)  # 예측 결과를 리스트에 추가

predictions  # 생성된 예측 리스트 출력

100%|██████████| 10/10 [00:43<00:00,  4.33s/it]


['The Trail Patrol Training course typically focuses on a variety of skills aimed at ensuring the safety, maintenance, and enjoyment of trails. While the specific curriculum can vary depending on the organization offering the course, some common skills taught include:\n\n1. **First Aid and CPR**: Basic first aid and CPR skills are essential for responding to medical emergencies on the trail.\n\n2. **Navigation and Map Reading**: Understanding how to read maps, use a compass, and navigate using GPS devices.\n\n3. **Trail Maintenance**: Techniques for maintaining trails, including erosion control, clearing debris, and repairing trail surfaces.\n\n4. **Communication Skills**: Effective communication with trail users, including conflict resolution and providing information about trail conditions and regulations.\n\n5. **Environmental Awareness**: Knowledge of local flora and fauna, Leave No Trace principles, and how to minimize human impact on natural environments.\n\n6. **Safety Protocols

In [5]:
from ragas.metrics import FactualCorrectness  # FactualCorrectness 메트릭 import
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset  # 평가 데이터셋 스키마와 단일 회차 샘플 import
from ragas import evaluate  # 평가 함수 import

samples = []  # 평가 샘플을 저장할 리스트 초기화
for question, ai_response, ground_truth in zip(questions, predictions, answers):  # 질문, 예측, 정답을 묶어서 반복
    sample = SingleTurnSample(
        user_input=question,  # 사용자 질문 설정
        response=ai_response,  # AI 응답 설정
        reference=ground_truth  # 참조 정답 설정
    )
    samples.append(sample)  # 평가 샘플 리스트에 추가

dataset = EvaluationDataset(samples)  # 평가 데이터셋 생성

factual_correctness_metric = FactualCorrectness(llm=evaluator_llm)  # LLM 평가를 위한 FactualCorrectness 메트릭 설정

results = evaluate(dataset, metrics=[factual_correctness_metric])  # 데이터셋과 메트릭을 사용해 평가 실행

results_df = results.to_pandas()  # 평가 결과를 DataFrame으로 변환
print(results_df.head())  # 평가 결과 상위 몇 개의 행 출력

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

                                          user_input  \
0  What are some of the skills taught in the Trai...   
1  Who was the original owner of the lot of items...   
2  What is the main objective of Humanity Road as...   
3  Who were the two convicted killers that escape...   
4  Who was the person that came to help when Isaa...   

                                            response  \
0  The Trail Patrol Training course typically foc...   
1  To accurately determine the original owner of ...   
2  Humanity Road is a non-profit organization tha...   
3  The two convicted killers who escaped from the...   
4  The question appears to be more related to a s...   

                                           reference  factual_correctness  
0  The course teaches the essential skills necess...                 0.25  
1  The original lot of items belonged to Lt. Neil...                 0.00  
2  The main objective of Humanity Road is to 'clo...                 0.00  
3  The two convicted k

In [6]:
print(results)  # 전체 평가 결과 출력

{'factual_correctness': 0.1690}


In [7]:
print(questions[0])  # 첫 번째 질문 출력

What are some of the skills taught in the Trail Patrol Training course?


In [8]:
print(predictions[0])  # 첫 번째 예측 출력

The Trail Patrol Training course typically focuses on a variety of skills aimed at ensuring the safety, maintenance, and enjoyment of trails. While the specific curriculum can vary depending on the organization offering the course, some common skills taught include:

1. **First Aid and CPR**: Basic first aid and CPR skills are essential for responding to medical emergencies on the trail.

2. **Navigation and Map Reading**: Understanding how to read maps, use a compass, and navigate using GPS devices.

3. **Trail Maintenance**: Techniques for maintaining trails, including erosion control, clearing debris, and repairing trail surfaces.

4. **Communication Skills**: Effective communication with trail users, including conflict resolution and providing information about trail conditions and regulations.

5. **Environmental Awareness**: Knowledge of local flora and fauna, Leave No Trace principles, and how to minimize human impact on natural environments.

6. **Safety Protocols**: Procedures

In [9]:
import numpy as np  # NumPy 모듈 import
from tqdm import tqdm  # 진행 상태 표시를 위한 tqdm 모듈 import

from utils import call_openai, get_embeddings, cosine_similarity  # 사용자 정의 모듈에서 필요한 함수 import

def retrieve_context(question, contexts):  # 질문에 가장 유사한 문맥을 찾기 위한 함수 정의
    question_embedding = get_embeddings([question], model='text-embedding-3-large')[0]  # 질문 임베딩 생성
    context_embeddings = get_embeddings(contexts, model='text-embedding-3-large')  # 문맥 임베딩 생성

    similarities = [cosine_similarity(question_embedding, context_embedding) for context_embedding in context_embeddings]  # 질문과 각 문맥 간의 유사도 계산

    most_relevant_index = np.argmax(similarities)  # 가장 높은 유사도를 가지는 문맥의 인덱스 찾기
    return contexts[most_relevant_index]  # 가장 관련성이 높은 문맥 반환

predictions = []  # 예측 결과를 저장할 리스트 초기화
for i in tqdm(range(len(questions))):  # 질문 수 만큼 반복하며 진행 상태 표시
    context = retrieve_context(questions[i], contexts[i])  # 질문에 가장 유사한 문맥 찾기
    prompt = f"""You are an expert in Finance. Please answer to the question given below. Use information given in Context appropriately.

Context:
{context}

Question:
{questions[i]}
"""  # 문맥과 질문을 기반으로 답변을 생성하는 프롬프트 생성
    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')  # OpenAI API 호출을 통해 예측 생성
    predictions.append(prediction)  # 예측 결과를 리스트에 추가

100%|██████████| 10/10 [00:43<00:00,  4.35s/it]


In [10]:
from ragas.metrics import FactualCorrectness  # FactualCorrectness 메트릭 import
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset  # 평가 데이터셋 스키마와 단일 회차 샘플 import
from ragas import evaluate  # 평가 함수 import

samples = []  # 평가 샘플을 저장할 리스트 초기화
for question, ai_response, ground_truth in zip(questions, predictions, answers):  # 질문, 예측, 정답을 묶어서 반복
    sample = SingleTurnSample(
        user_input=question,  # 사용자 질문 설정
        response=ai_response,  # AI 응답 설정
        reference=ground_truth  # 참조 정답 설정
    )
    samples.append(sample)  # 평가 샘플 리스트에 추가

dataset = EvaluationDataset(samples)  # 평가 데이터셋 생성

factual_correctness_metric = FactualCorrectness(llm=evaluator_llm)  # LLM 평가를 위한 FactualCorrectness 메트릭 설정

results = evaluate(dataset, metrics=[factual_correctness_metric])  # 데이터셋과 메트릭을 사용해 평가 실행

results_df = results.to_pandas()  # 평가 결과를 DataFrame으로 변환
print(results_df.head())  # 평가 결과 상위 몇 개의 행 출력

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

                                          user_input  \
0  What are some of the skills taught in the Trai...   
1  Who was the original owner of the lot of items...   
2  What is the main objective of Humanity Road as...   
3  Who were the two convicted killers that escape...   
4  Who was the person that came to help when Isaa...   

                                            response  \
0  It appears that the provided context is incomp...   
1  I'm sorry, but the context provided ("L") does...   
2  The main objective of Humanity Road as a non-p...   
3  The two convicted killers who escaped from an ...   
4  The context provided does not contain any info...   

                                           reference  factual_correctness  
0  The course teaches the essential skills necess...                 0.43  
1  The original lot of items belonged to Lt. Neil...                 0.00  
2  The main objective of Humanity Road is to 'clo...                 0.33  
3  The two convicted k

In [11]:
print(results)  # 전체 평가 결과 출력

{'factual_correctness': 0.2810}


In [12]:
print(predictions[0])  # 첫 번째 예측 출력

It appears that the provided context is incomplete ("T"). However, based on general knowledge about Trail Patrol Training courses, I can provide an overview of the skills typically taught in such programs. 

Trail Patrol Training courses often cover a variety of skills to ensure that participants are well-prepared to manage and maintain trails effectively. Some of the key skills taught in these courses may include:

1. **Trail Maintenance and Repair**: Techniques for maintaining and repairing trails, including erosion control, drainage management, and vegetation management.

2. **Safety and First Aid**: Basic first aid and emergency response skills to handle injuries or emergencies that may occur on the trail.

3. **Navigation and Map Reading**: Skills for reading maps, using GPS devices, and navigating trails to ensure patrols can effectively cover their assigned areas.

4. **Communication Skills**: Effective communication techniques for interacting with trail users, reporting issues,

In [13]:
print(answers[0])  # 첫 번째 정답 출력

The course teaches the essential skills necessary to be a trail patrol member and to provide a reassuring presence on the trail while teaching safety and environmental responsibility.
